# PnL quantheano module

In [1]:
import numpy as np
import theano
import theano.tensor as T

import matplotlib.pyplot as plt
%matplotlib inline

In [79]:
_buying_power = theano.shared(10. ** 6)
_commis_per_share = theano.shared(0.01)
_nb_top = theano.shared(5)
_nb_bottom = theano.shared(5)

In [80]:
def strategy_returns(shares, abs_returns):
    """
    RETURN
    Returns theano function which allows you to calculate pnl diff day by day($)
        providing matrix shares in target portfolio and matrix of abs returns over time (of equal shape)
    
    -----
    INPUTS
    shares: theano tensor var. Matrix of shares supposed to be constructed as
        follows: (time, symbol) axis. Shares at t means how many shares you own at t.
    abs_returns: theano tensor var. Matrix abs_returns supposed to be constructed as
        follows: (time, symbol) axis. Return at t is price_{t} - price_{t-1}.       
    
    -----
    EXAMPLE:
 
    import time 

    shares = T.matrix(name='(time, symbol) shares matrix')
    abs_returns = T.matrix(name='(time, symbol) abs returns matrix')
    # compile pnl func
    pnl_func = theano.function([shares, abs_returns], strategy_returns(shares, abs_returns), allow_input_downcast=True)
    # data for an example
    abs_returns_val = np.random.randn(10 ** 6, 2) + .01
    shares_val = np.ones((10 ** 6, 2))

    %time pnl_arr = pnl_func(shares_val, abs_returns_val)
    plt.figure()
    plt.title('PnL example plot')
    plt.grid()
    plt.plot(pnl_arr)

    
    """
    commis = _commis_per_share*T.abs_(shares[1:] - shares[:-1]).sum(axis=1)
    return (shares[:-1] * abs_returns[1:]).sum(axis=1) -\
            T.maximum(commis, 1.) * (commis > 0).astype('float32')


# compile func
_shares = T.matrix(name='(time, symbol) shares matrix')
_abs_returns = T.matrix(name='(time, symbol) abs returns matrix')
strategy_returns_fun = theano.function([_shares, _abs_returns], strategy_returns(_shares, _abs_returns), allow_input_downcast=True)

In [89]:
"""
# tests
rt = np.random.randn(5, 2)
rt
shares = np.array([[1.5, 0., 2., 1., 1.], [0., 1., 1., 1., 1.]]).transpose()
shares[:-1]
strategy_returns_fun(shares, rt)
"""

'\n# tests\nrt = np.random.randn(5, 2)\nrt\nshares = np.array([[1.5, 0., 2., 1., 1.], [0., 1., 1., 1., 1.]]).transpose()\nshares[:-1]\nstrategy_returns_fun(shares, rt)\n'

In [4]:
def pnl(shares, abs_returns):
    """
    RETURN
    Returns theano function which allows you to calculate pnl ($)
        providing matrix shares in target portfolio and matrix of abs returns over time (of equal shape)
    
    -----
    INPUTS
    shares: theano tensor var. Matrix of shares supposed to be constructed as
        follows: (time, symbol) axis. Shares at t means how many shares you own at t.
    abs_returns: theano tensor var. Matrix abs_returns supposed to be constructed as
        follows: (time, symbol) axis. Return at t is price_{t} - price_{t-1}.       
    
    -----
    EXAMPLE:
 
    import time 

    shares = T.matrix(name='(time, symbol) shares matrix')
    abs_returns = T.matrix(name='(time, symbol) abs returns matrix')
    # compile pnl func
    pnl_func = theano.function([shares, abs_returns], pnl(shares, abs_returns), allow_input_downcast=True)
    # data for an example
    abs_returns_val = np.random.randn(10 ** 6, 2) + .01
    shares_val = np.ones((10 ** 6, 2))

    %time pnl_arr = pnl_func(shares_val, abs_returns_val)
    plt.figure()
    plt.title('PnL example plot')
    plt.grid()
    plt.plot(pnl_arr)

    
    """
    return strategy_returns(shares, abs_returns).cumsum()

# compile func
pnl_fun = theano.function([_shares, _abs_returns], pnl(_shares, _abs_returns), allow_input_downcast=True)

In [5]:
def market_value(shares, margin_per_share):
    """
    RETURN
    Returns theano expression which allows you to calculate portfolio margin ($)
        providing matrix of shares in target portfolio and matrix of margin per share over time (of equal shape). 
    
    -----
    COMMENT
    Margin per share for a stock is just its not-adjusted price
    -----
    INPUTS
    shares: theano tensor var. Matrix of shares supposed to be constructed as
        follows: (time, symbol) axis. Shares at t means how many shares you own at t.
    margin_per_share: theano tensor var. Matrix of margin per share supposed to be constructed as
        follows: (time, symbol) axis. Price of the share at t.       

    -----
    """ 
    return (T.abs_(shares) * margin_per_share).sum(axis=1)


# compile func
_margin_per_share = T.matrix(name='(time, symbol) margin per share matrix')
market_value_fun = theano.function([_shares, _margin_per_share], market_value(_shares, _margin_per_share), allow_input_downcast=True)

In [6]:
def quality_to_share_robust(quality, margin_per_share, mask):
    """
    RETURN
    Returns theano expression which allows you to calculate target number of shares in portfolio based on quality matrix.
    -----
    INPUTS
    quality : quality matrix - theano tensor
    margin_per_share : theano tensor var. Matrix of margin per share supposed to be constructed as
        follows: (time, symbol) axis. Price of the share at t.     
    buying_power : theanp shared
    nb_top : top nb by quality will be given long of equal mv
    nb_bottom : bottom nb by quality will be given short of equal mv
    -----
    
    EXAMPLE
    bp = theano.shared(10000.)
    quality = T.matrix(name='(time, symbol) shares matrix')
    margin_per_share = T.matrix(name='(time, symbol) margin_per_share matrix')
    # compile pnl func
    quality_to_share_robust_func = theano.function([quality, margin_per_share],\
                                     quality_to_share_robust(quality, margin_per_share),\
                                                       allow_input_downcast=True)

    quality_val = np.random.randn(10, 10)
    margin_val = np.ones((10, 10))
    print(quality_val)
    print(quality_to_share_robust_func(quality_val, margin_val))

    """
    
        
    argsort_Q = quality.argsort().argsort()
    alphas = ((argsort_Q > quality.shape[1]-_nb_top-1).astype('float32')/_nb_top -\
              (argsort_Q < _nb_bottom).astype('float32')/_nb_bottom) 
    shares = .5 * alphas * _buying_power * T.inv(margin_per_share)
    shares = mask.astype('float32') * shares
    return T.floor(shares)


# compile func
_mask = T.matrix(name='(time, symbol) mask matrix')
_quality = T.matrix(name='(time, symbol) quality matrix')
quality_to_share_robust_fun = theano.function([_quality, _margin_per_share, _mask],\
                                        quality_to_share_robust(_quality, _margin_per_share, _mask), allow_input_downcast=True)

In [7]:
def quality_to_share_soft(quality, margin_per_share, mask, epsilon=1e-5):
    """
    RETURN
    Returns theano expression which allows you to calculate target number of shares in portfolio based on quality matrix.
    -----
    INPUTS
    quality : quality matrix - theano tensor
    margin_per_share : theano tensor var. Matrix of margin per share supposed to be constructed as
        follows: (time, symbol) axis. Price of the share at t.     
    buying_power : theano shared
    -----
    
    EXAMPLE
    bp = theano.shared(10000.)
    quality = T.matrix(name='(time, symbol) shares matrix')
    margin_per_share = T.matrix(name='(time, symbol) margin_per_share matrix')
    # compile pnl func
    quality_to_share_soft_func = theano.function([quality, margin_per_share],\
                                     quality_to_share_soft(quality, margin_per_share),\
                                                       allow_input_downcast=True)

    quality_val = np.random.randn(10, 10)
    margin_val = np.ones((10, 10))
    print(quality_val)
    print(np.abs(quality_to_share_soft_func(quality_val, margin_val)).sum(axis=1, keepdims=True))


    """
    
    quality = quality - quality.mean(axis=1, keepdims=True)
    alphas = quality / (epsilon + T.abs_(quality).sum(axis=1, keepdims=True))    
    shares = (.5 * alphas * _buying_power * T.inv(margin_per_share))
    shares = mask.astype('float32') * shares
    return T.floor(shares)

# compile func
quality_to_share_soft_fun = theano.function([_quality, _margin_per_share, _mask],\
                                        quality_to_share_soft(_quality, _margin_per_share, _mask), allow_input_downcast=True)

### Examples:

In [8]:
"""
# example
bp = theano.shared(10000.)
quality = T.matrix(name='(time, symbol) shares matrix')
margin_per_share = T.matrix(name='(time, symbol) margin_per_share matrix')
# compile pnl func
quality_to_share_soft_func = theano.function([quality, margin_per_share],\
                                 quality_to_share_soft(quality, margin_per_share, buying_power=bp),\
                                                   allow_input_downcast=True)

quality_val = np.random.randn(10, 10)
margin_val = np.ones((10, 10))
print(quality_val)
print(np.abs(quality_to_share_soft_func(quality_val, margin_val)).sum(axis=1))
"""

"\n# example\nbp = theano.shared(10000.)\nquality = T.matrix(name='(time, symbol) shares matrix')\nmargin_per_share = T.matrix(name='(time, symbol) margin_per_share matrix')\n# compile pnl func\nquality_to_share_soft_func = theano.function([quality, margin_per_share],                                 quality_to_share_soft(quality, margin_per_share, buying_power=bp),                                                   allow_input_downcast=True)\n\nquality_val = np.random.randn(10, 10)\nmargin_val = np.ones((10, 10))\nprint(quality_val)\nprint(np.abs(quality_to_share_soft_func(quality_val, margin_val)).sum(axis=1))\n"

In [9]:
"""
# example
import time 

shares = T.matrix(name='(time, symbol) shares matrix')
abs_returns = T.matrix(name='(time, symbol) abs returns matrix')
# compile pnl func
pnl_func = theano.function([shares, abs_returns], pnl(shares, abs_returns), allow_input_downcast=True)
# data for an example
abs_returns_val = np.random.randn(10 ** 6, 2) + .01
shares_val = np.ones((10 ** 6, 2))

%time pnl_arr = pnl_func(shares_val, abs_returns_val)
plt.figure()
plt.title('PnL example plot')
plt.grid()
plt.plot(pnl_arr)
"""

"\n# example\nimport time \n\nshares = T.matrix(name='(time, symbol) shares matrix')\nabs_returns = T.matrix(name='(time, symbol) abs returns matrix')\n# compile pnl func\npnl_func = theano.function([shares, abs_returns], pnl(shares, abs_returns), allow_input_downcast=True)\n# data for an example\nabs_returns_val = np.random.randn(10 ** 6, 2) + .01\nshares_val = np.ones((10 ** 6, 2))\n\n%time pnl_arr = pnl_func(shares_val, abs_returns_val)\nplt.figure()\nplt.title('PnL example plot')\nplt.grid()\nplt.plot(pnl_arr)\n"